In [ ]:
import datasets

In [ ]:
dataset1 = datasets.load_dataset('HAERAE-HUB/KOREAN-WEBTEXT', split='train')
dataset1

In [ ]:
dataset2 = datasets.load_dataset('blueapple8259/c4-ko-cleaned-2', split='train')
dataset2

In [ ]:
dataset3 = datasets.load_dataset('HAERAE-HUB/KOREAN-SyntheticText-1.5B', split='train')
dataset3

In [ ]:
# maywell/korean_textbooks 데이터셋에서 분류기로 3점 이상의 데이터만 수집
dataset4 = datasets.load_dataset("devngho/korean-textbooks-edu", name="scored_over_3", split="train")
dataset4

In [ ]:
from datasets import concatenate_datasets

combined_dataset = concatenate_datasets([dataset1, dataset2, dataset3, dataset4])
combined_dataset

In [ ]:
seen = set()
def filter_duplicates(example):
    text = example["text"]
    if text in seen:
        return False
    seen.add(text)
    return True

unique_dataset = combined_dataset.filter(filter_duplicates)
unique_dataset

In [ ]:
# text 컬럼만 남기고 나머지 컬럼 제거
columns_to_remove = [col for col in unique_dataset.column_names if col != "text"]
text_only_dataset = unique_dataset.remove_columns(columns_to_remove)
text_only_dataset

In [ ]:
# text안에 내용이 100글자 아래인 경우에 대해서 제거
def filter_short_texts(example):
    return len(example["text"]) >= 100

filtered_dataset = text_only_dataset.filter(filter_short_texts)
filtered_dataset

In [ ]:
shuffled_dataset = filtered_dataset.shuffle(seed=5768112)

In [ ]:
shuffled_dataset.push_to_hub("minpeter/pretrain-korean-dedup", split="train")